In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyautogui

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pynput==1.7.6

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import util
import pyautogui
from pynput.mouse import Button, Controller
import random 
# import time

In [5]:
# pip install opencv-python

In [6]:
# pip install opencv-contrib-python

In [7]:
pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
aiofiles                     23.2.1
altair                       5.4.1
annotated-types              0.7.0
anyio                        4.5.2
asgiref                      3.8.1
asttokens                    3.0.0
astunparse                   1.6.3
attrs                        25.3.0
backcall                     0.2.0
backports.zoneinfo           0.2.1
beautifulsoup4               4.13.3
blinker                      1.8.2
cachetools                   5.5.2
certifi                      2025.1.31
cffi                         1.17.1
charset-normalizer           3.4.1
click                        8.1.8
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.1.1
cycler                       0.12.1
debugpy                      1.8.13
decorator                    5.2.1
Django                       4.2.16
et_xmlfile                   2.0.0
e

In [8]:
screen_width, screen_height = pyautogui.size()
mouse = Controller()

mphands = mp.solutions.hands
hands = mphands.Hands(
    static_image_mode = False, # no image hands needed to detect 
    model_complexity = 1,
    min_detection_confidence = 0.7, # in order to identify a thing in display as a hand  the accuracy must be more min 70%
    min_tracking_confidence = 0.7,  # in order to track the movement of hand min accurac of 70 % required 
    max_num_hands = 1 # is display contain more number of hand it need to recognize only one 
)

In [9]:
def move_mouse(index_finger_tip):
    if index_finger_tip is not None:
        x = int(index_finger_tip.x * 1000)  # the position value will be in a between a 0 and 1 but we need  value like less than 50 like that so we multiply it by 1000
        y = int(index_finger_tip.y * 1000)
        pyautogui.moveTo(x,y)

In [10]:
def find_finger_tip(processed):
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        return hand_landmarks.landmark[mphands.HandLandmark.INDEX_FINGER_TIP]
    return None

In [11]:
def is_left_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 30  and 
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) > 50 and 
            util.get_angle(landmarks_list[17], landmarks_list[18], landmarks_list[20]) > 90 and
            # thumb_indextip_dist > 40 and
            thumb_index_dist > 75
            )


def is_right_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 50 and 
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 30  and 
            util.get_angle(landmarks_list[17], landmarks_list[18], landmarks_list[20]) > 90 and
            thumb_index_dist > 75
            )

def is_double_click(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and 
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and 
            thumb_index_dist > 75
            )

def is_screenshot(landmarks_list, thumb_index_dist):
    return (util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) < 50 and 
            util.get_angle(landmarks_list[9], landmarks_list[10], landmarks_list[12]) < 50 and 
            thumb_index_dist < 75
            )

# def is_drag(landmarks_list, thumb_mid_dist):
#     return (util.get_angle(landmarks_list[17], landmarks_list[18], landmarks_list[20]) > 90 and
#             util.get_angle(landmarks_list[5], landmarks_list[6], landmarks_list[8]) > 50 and 
#             thumb_mid_dist < 30
            # )

In [12]:
def detect_gestures(frame, landmarks_list, processed):
    if len(landmarks_list)>=21:
        index_finger_tip = find_finger_tip(processed)
        thumb_index_dist = util.get_distance([landmarks_list[4],landmarks_list[5]])
        # thumb_mid_dist = util.get_distance([landmarks_list[4],landmarks_list[8]])
        # thumb_indextip_dist = util.get_distance([landmarks_list[8],landmarks_list[4]])
        # thumb_midtip_dist = util.get_distance([landmarks_list[12],landmarks_list[1]])

        if thumb_index_dist < 30 and util.get_angle(landmarks_list[5],landmarks_list[6],landmarks_list[8]) > 90 :
            move_mouse(index_finger_tip) 

        
        # left click
        elif is_left_click(landmarks_list, thumb_index_dist): 
            mouse.press(Button.left)
            mouse.release(Button.left)

        # right click
        elif is_right_click(landmarks_list, thumb_index_dist):
            mouse.press(Button.right)
            mouse.release(Button.right)


        # double click
        elif is_double_click(landmarks_list, thumb_index_dist): 
            pyautogui.doubleClick()



        # screenshot
        elif is_screenshot(landmarks_list, thumb_index_dist):
            im1 = pyautogui.screenshot()
            label = random.randint(1, 1000)
            im1.save(f'my_screenshot_{label}.png')

        # elif is_drag(landmarks_list,thumb_mid_dist):
        #      if index_finger_tip is not None:
        #         x = int(index_finger_tip.x * 1000)  # the position value will be in a between a 0 and 1 but we need  value like less than 50 like that so we multiply it by 1000
        #         y = int(index_finger_tip.y * 1000)
        #         pyautogui.dragTo(x,y)
            


In [13]:
pyautogui.FAILSAFE = False

In [ ]:
def  main():
    cap = cv2.VideoCapture(0)
    draw = mp.solutions.drawing_utils  # to draw a hand marks and all
    try:
        while cap.isOpened():
            ret,frame = cap.read()   #ret is returning a boolean value if abel to capture a video it will return true else false
            
            if not ret:    #if ret is false
                print ('not abel to open a video')
            frame = cv2.flip(frame,1) # mirror the video that will visible like we are viewing a camera
            framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # to convert the color from bgr to rgb
            
            processed = hands.process(framergb)  # to process the color changed hand and store in a variable
            landmarks_list = []
            
            if processed.multi_hand_landmarks:   # if anything is processed then 
                hand_landmarks = processed.multi_hand_landmarks[0]   # select any one of the processed hand land marks and store it in a variable
                draw.draw_landmarks(frame, hand_landmarks, mphands.HAND_CONNECTIONS) # after identifying a landmarks on a hand join them 
                 
                for lm in hand_landmarks.landmark:
                    landmarks_list.append((lm.x, lm.y))

                
                # print(landmarks_list)
                
            detect_gestures(frame, landmarks_list, processed)  #calling a function
                
            cv2.imshow('Frame',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  #wait for 1sec for each frame and 0xff says that a keyboard input : if keyboard input is q then video capturing will end
                break
                
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
if __name__ == '__main__':  #file wont run any where except this main file
    main()

In [ ]:
brd = int(input("enter your DOB : "))
cur = int(2024)
if brd < 2024 :
    age = cur - brd
    print("the age is : ", age)
else :
    print("invalid DOB ! ")